In [1]:
import numpy as np
import mnist
import pickle
from PIL import Image

## 1. Load MNIST dataset
- 책에서 나온 라이브러리는 사용이 안되는 것 같아 다른 라이브러리를 사용 (다행히 MNIST를 제공하는 라이브러리는 많음)
- chapter 4, 5 모두 mnist를 사용하는데 매번 라이브러리에서 불러오기 귀찮으니 pickle로 저장해둔다.
- train/test, image/label로 나눈다. 
- 현재 shape는 (size, m, n)으로 m*n matrix가 size개만큼 있는 상태. 우선 책과 shape를 맞추기 위해 m X n matrix를 길이가 m*n인 vector로 변환

In [151]:
train_images = mnist.train_images()
train_labels = mnist.train_labels()
test_images = mnist.test_images()
test_labels = mnist.test_labels()
temp = [train_images, train_labels, test_images, test_labels] 

In [152]:
#mnist 데이터 pickle로 저장
with open("mnist_dataset.txt", mode="wb") as fp : 
    dataset = [train_images, train_labels, test_images, test_labels] 
    pickle.dump(dataset, fp)
    
#mnist 데이터 pickle로 로드
temp = []
with open("mnist_dataset.txt", mode="rb") as fp : 
    temp = pickle.load(fp)
train_images, train_labels, test_images, test_labels = temp

In [155]:
print(train_images.shape)
print(train_labels.shape)

(60000, 28, 28)
(60000,)


In [161]:
#reshape (28, 28) -> (784)
temp[0] = np.reshape(temp[0], [temp[0].shape[0], -1])
temp[2] = np.reshape(temp[2], [temp[2].shape[0], -1])
x_train, t_train, x_test, t_test = temp

In [5]:
#shape check
x_train.shape

(60000, 784)

In [6]:
#checking data
img = x_train[0]
label = x_train[0]
print(label)

[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   3  18  18  18 126 136 175  26 166 255
 247 127   0   0   0   0   0   0   0   0   0   0   0   0  30  36  94 154
 170 253 253 253 253 253 225 172 253 242 195  64   0   0   0   0   0   0
   0   0   0   0   0  49 238 253 253 253 253 253 253 253 253 251  93  82
  82  56  39   0   0   0   0   0   0   0   0   0   0   0   0  18 219 253
 253 253 253 253 198 182 247 241   0   0   0   0   

In [7]:
print(img.shape)
img = img.reshape(28, 28)
print(img.shape)

(784,)
(28, 28)


In [8]:
#function showing label as image
def img_show(img) : 
    pil_img = Image.fromarray(np.uint8(img))
    pil_img.show()

In [9]:
#show data[0] label as image
img_show(img)

<br><hr><br>

## 2. Load function 
- 앞서 작성한 loss & activation function을 가져온다. (나중에 .py로 옮겨서 라이브러리처럼 사용할 것)

In [10]:
#sigmoid_function : 주어진 값을 0~1 사이의 값으로 변환해주는 S자 모양의 함수. 0보다 크면 0.5보다 크게, 0보다 작으면 0.5보다 작게 변환된다.
def sigmoid_function(x) : 
    return (1/(1+np.exp(-x)))

In [11]:
#identity_function : 마지막 출력 함수에서 주로 사용하는 활성화 함수. 형식을 맞추기 위해 사용될 뿐 실제 의미는 없다.
def identity_function(x) :
    return x

In [12]:
#softmax_function
def softmax_function(x) : 
    max_by_row = np.amax(x, axis=1)
    max_by_row = np.reshape(max_by_row, [max_by_row.shape[0],1])
    nom = np.exp(x-max_by_row)
    denom = np.sum(np.exp(x-max_by_row), axis=1)
    denom = np.reshape(denom, [denom.shape[0],1])
    return nom/denom

In [13]:
#CEE (cross_entropy_error, 교차엔트로피오차)
def CEE(y_p, y_t) : #y_p: predicted vector, y_t: actual_vector
    if y_t.ndim == 1 : 
        y_t = y_t.reshape(1, y_t.size)
        y_p = y_p.reshape(1, y_p.size)
        
    delta = 1e-7
    return -1/y_p.shape[0]*np.sum(y_t*np.log(y_p + delta)) #y_p가 0이 되어 log 계산시 -inf가 나오는 현상 방지

In [14]:
#ㅜnumerical differentian function (수치 미분 함수)
def numerical_gradient(f, x) : #편미분은 특정 변수에 대한 미분을 의미했는데, 
    #gradient는 각각의 변수에 대해 전부 편미분하여 그 값을 numpy array형태로 반환한다. 실제 내부 코드는 partial_diff와 크게 다르지 않다.
    h = 1e-4 #0.0001
    grad = np.zeros_like(x)
    
    for idx in range(0, x.size) : 
        temp = x[idx]
        
        x[idx] = temp + h
        fxh1 = f(x)
        x[idx] = temp - h 
        fxh2 = f(x)
        
        grad[idx] = (fxh1-fxh2)/(2*h)
        x[idx] = temp
        
    return grad

<br><hr><br>

## 3. Modeling MLP
- tensorflow와 마찬가지로, numpy로 구현하는 MLP는 간선에 들어갈 가중치와 노드들만 정의해두고 간선의 연결(굳이 이름 붙이자면, 노드와 노드 간의 연산)을 수식(이 경우, "matmul(X*W)+b")을 통해 정의한다.
- init_network()는 간선의 가중치 값을 담는 사전인데, 현재는 hidden_layer의 개수와 layer 내의 node수는 수동으로 정해줘야 한다. (이후 용도에 따라 parameter로 받도록 수정 가능할 듯)

In [15]:
def init_network() : #간선의 가중치 값을 담는 사전
    network = {} 
    network["W1"] = np.random.random_sample((784,50))
    network["b1"] = np.random.random_sample((50,))
    network["W2"] = np.random.random_sample((50,100))
    network["b2"] = np.random.random_sample((100,))
    network["W3"] = np.random.random_sample((100,10))
    network["b3"] = np.random.random_sample((10,))
    
    return network

In [16]:
def predict(network, x) : 
    W1, W2, W3 = network["W1"], network["W2"], network["W3"]
    b1, b2, b3 = network["b1"], network["b2"], network["b3"]
    
    a1 = np.dot(x, W1) + b1
    z1 = sigmoid_function(a1)
    a2 = np.dot(z1, W2) + b2
    z2 = sigmoid_function(a2) 
    a3 = np.dot(z2, W3) + b3
    y = identity_function(a3)
    
    return y

In [17]:
network = init_network() #네트워크 초기화

In [18]:
x, t = x_test, t_test

In [19]:
accuracy_cnt = 0 
batch_size = 32
for i in range(0, len(x), batch_size) : #mini_batch 처리
    if i+batch_size >= len(x) :
        x_batch = x[i:len(x)] #마지막 부분은 예외처리 (list index range)
    else : 
        x_batch = x[i:i+batch_size] 
        
    y_batch = predict(network, x_batch)
    p = np.argmax(y_batch, axis=1)
    t_batch = np.reshape(t[i:i+batch_size], [-1,len(t[i:i+batch_size])]) #label이 matrix이므로 flatten하여 vector로 변환
    accuracy_cnt += np.sum(p==t_batch)

print("Accuracy:", str(float(accuracy_cnt)/len(x)))

Accuracy: 0.0892


In [20]:
print(accuracy_cnt)

892


In [122]:
class MLP : 
    def __init__(self, input_size, hidden_size_list, output_size, weight_init_std = 0.01) : 
        #가중치 초기화
        self.params = {}
        self.hidden_len = len(hidden_size_list)
        for i in range(1, self.hidden_len+2) : #hidden_layer의 수 + output_layer
            if i==self.hidden_len+1 : 
                self.params["W"+str(i)] = weight_init_std * np.random.randn(hidden_size_list[i-2], output_size) 
                self.params["b"+str(i)] = np.zeros(output_size)
            elif i==1 : 
                self.params["W"+str(i)] = weight_init_std * np.random.randn(input_size, hidden_size_list[i-1]) 
                self.params["b"+str(i)] = np.zeros(hidden_size_list[i-1])
            else :     
                self.params["W"+str(i)] = weight_init_std * np.random.randn(hidden_size_list[i-2], hidden_size_list[i-1]) 
                self.params["b"+str(i)] = np.zeros(hidden_size_list[i-1])
            
    
    def predict(self, x) : #x: 입력 데이터
        a = 0
        z = x
        
        i = 1
        for i in range(1, self.hidden_len+1) : 
            a = np.dot(z, self.params["W"+str(i)]) + self.params["b"+str(i)]
            z = sigmoid_function(a)
        i+=1
        a = np.dot(z, self.params["W"+str(i)]) + self.params["b"+str(i)]
        y = softmax_function(a)
    
        return y
    
    
    def loss(self, x, t) : #x: 입력 데이터, t: 정답 레이블
        y = self.predict(x)
        return CEE(y, t)
    
    
    def accuracy(self, x, t) : #x: 입력 데이터, t: 정답 레이블
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y==t)/float(x.shape[0])
        return accuracy
    
    
    #ㅜnumerical differentian function for weigths (수치 미분 함수)
    def weight_numerical_gradient(self, f, x) : #편미분은 특정 변수에 대한 미분을 의미했는데, 
        #gradient는 각각의 변수에 대해 전부 편미분하여 그 값을 numpy array형태로 반환한다. 실제 내부 코드는 partial_diff와 크게 다르지 않다.
        h = 1e-4 #0.0001
        grad = np.zeros_like(x)

        for i in range(0, x.shape[0]) : 
            for j in range(0, x.shape[1]) : 
                temp = x[i][j]

                x[i][j] = temp + h
                fxh1 = f(x[i][j])
                x[i][j] = temp - h 
                fxh2 = f(x[i][j])

                grad[i][j] = (fxh1-fxh2)/(2*h)
                x[i][j] = temp

        return grad
    
    
    #ㅜnumerical differentian function for bias (수치 미분 함수)
    def bias_numerical_gradient(self, f, x) : #편미분은 특정 변수에 대한 미분을 의미했는데, 
        #gradient는 각각의 변수에 대해 전부 편미분하여 그 값을 numpy array형태로 반환한다. 실제 내부 코드는 partial_diff와 크게 다르지 않다.
        h = 1e-4 #0.0001
        grad = np.zeros_like(x)

        for i in range(0, x.shape[0]) : 
            temp = x[i]

            x[i] = temp + h
            fxh1 = f(x[i])
            x[i] = temp - h 
            fxh2 = f(x[i])

            grad[i] = (fxh1-fxh2)/(2*h)
            x[i] = temp

        return grad
    
    
    #수치미분 함수를 이용해 각 가중치에 대응하는 gradient 사전을 얻음
    def get_gradient(self, x, t) :
        loss_value = self.loss(x, t)
        loss_W = lambda W: loss_value

        grads = {}
        for i in range(1, self.hidden_len+2) : #hidden_layer의 수 + output_layer
            grads["W"+str(i)] = self.weight_numerical_gradient(loss_W, self.params["W"+str(i)])
            grads["b"+str(i)] = self.bias_numerical_gradient(loss_W, self.params["b"+str(i)])

        return grads

<br><hr><br>

## 4. Training MLP model

In [167]:
x = np.random.rand(100,784)
t = np.random.rand(100,10)
y = net.predict(x)

In [185]:
#mnist 데이터 pickle로 로드
temp = []

with open("mnist_dataset.txt", mode="rb") as fp : 
    temp = pickle.load(fp)
    temp[0] = np.reshape(temp[0], [temp[0].shape[0], -1])
    temp[2] = np.reshape(temp[2], [temp[2].shape[0], -1])
    
    nb_classes = 10
    temp[1] = np.array([temp[1]]).reshape(-1)
    temp[1] = np.eye(nb_classes)[temp[1]]
    temp[3] = np.array([temp[3]]).reshape(-1)
    temp[3] = np.eye(nb_classes)[temp[3]]
    
x_train, y_train, x_test, y_test = temp

In [123]:
net = MLP(input_size=x_train.shape[1], hidden_size_list=[50,100], output_size=10)

In [129]:
grad = net.get_gradient(x, t)

In [135]:
for x in net.params.keys() : 
    print(x,"_weights : ",net.params[x].shape,"\t",x,"_gradients : ",grad[x].shape, sep="" )

W1_weights : (784, 50)	W1_gradients : (784, 50)
b1_weights : (50,)	b1_gradients : (50,)
b3_weights : (10,)	b3_gradients : (10,)
W3_weights : (100, 10)	W3_gradients : (100, 10)
b2_weights : (100,)	b2_gradients : (100,)
W2_weights : (50, 100)	W2_gradients : (50, 100)


In [191]:
#setting hyper parameter
iters_num = 1000
train_size = x_train.shape[0]
batch_size = 2**7
learning_rate = 0.1
iter_per_epoch = int(max(train_size/batch_size, 1))

In [192]:
iter_per_epoch

468

In [188]:
#result
train_loss_list = [] 
train_acc_list = []
test_acc_list = []

In [198]:
for i in range(0, iters_num) : 
    print("epoch:",i)
    '''
    #순차적으로 mini-batch
    if (i+1)*batch_size > train_size : 
        x_batch = x_train[train_size-i*batch_size:]
        y_batch = y_train[train_size-i*batch_size:]
    else : 
        x_batch = x_train[i*batch_size:(i+1)*batch_size]
        y_batch = y_train[i*batch_size:(i+1)*batch_size]
    '''
    
    #복원 추출로 랜덤 mini-batch
    batch_sample = np.random.choice(train_size, batch_size) 
    x_batch = x_train[batch_sample]
    y_batch = y_train[batch_sample]
    
    grad = net.get_gradient(x_batch, y_batch)
    
    #weights update (learning)
    for key in net.params.keys() : 
        net.params[key] -= learning_rate*grad[key]
        
    #learning result check
    loss = net.loss(x_batch, y_batch) 
    train_loss_list.append(loss)
    
    if i%iter_per_epoch == 0 : 
        train_acc = net.accuracy(x_train, y_train)
        test_acc = net.accuracy(x_test, y_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("epoch: %5d]]] loss: %f\ttrai acc: %f, test acc: %f"%(i, loss, train_acc, test_acc))

epoch: 0
epoch:     0]]] loss: 2.297773	trai acc: 0.112367, test acc: 0.113500
epoch: 1
epoch: 2
epoch: 3
epoch: 4
epoch: 5
epoch: 6
epoch: 7
epoch: 8
epoch: 9
epoch: 10
epoch: 11
epoch: 12
epoch: 13
epoch: 14
epoch: 15
epoch: 16
epoch: 17
epoch: 18
epoch: 19
epoch: 20
epoch: 21
epoch: 22
epoch: 23
epoch: 24
epoch: 25
epoch: 26
epoch: 27
epoch: 28
epoch: 29
epoch: 30
epoch: 31
epoch: 32
epoch: 33
epoch: 34
epoch: 35
epoch: 36
epoch: 37
epoch: 38
epoch: 39
epoch: 40
epoch: 41
epoch: 42
epoch: 43
epoch: 44
epoch: 45
epoch: 46
epoch: 47
epoch: 48
epoch: 49
epoch: 50
epoch: 51
epoch: 52
epoch: 53
epoch: 54
epoch: 55
epoch: 56
epoch: 57
epoch: 58
epoch: 59
epoch: 60
epoch: 61
epoch: 62
epoch: 63
epoch: 64
epoch: 65
epoch: 66
epoch: 67
epoch: 68
epoch: 69
epoch: 70
epoch: 71
epoch: 72
epoch: 73
epoch: 74
epoch: 75
epoch: 76
epoch: 77
epoch: 78
epoch: 79
epoch: 80
epoch: 81
epoch: 82
epoch: 83
epoch: 84
epoch: 85
epoch: 86
epoch: 87
epoch: 88
epoch: 89
epoch: 90
epoch: 91
epoch: 92
epoch: 93


KeyboardInterrupt: 

In [178]:
x_train.shape

(60000, 784)

In [179]:
y_train.shape

(60000, 10)

In [180]:
x_test.shape

(10000, 784)

In [181]:
y_test.shape

(10000, 10)